In [1]:
#Подготовим рабочую среду для дальнейшего задания

#! pip3 install pycodestyle flake8 pycodestyle_magic
#%load_ext pycodestyle_magic
#%pycodestyle_on
#! pip3 install gensim
#! pip3 install sklearn
import re
import gensim
import logging
import nltk.data
from nltk import tokenize
from pymystem3 import Mystem
m = Mystem()
import pandas as pd
import urllib.request
from nltk.corpus import stopwords
from gensim.models import word2vec
from gensim.models import KeyedVectors
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
#Прочитаем  файл с книгой. 
#Преобразуем текст в формат: только леммы, без знаков препинания, все в нижнем регистре,каждое предложение на отдельной строчке. 
#Создадим новый файл, в который запишем полученные предложения.

with open('dostoevsky.txt', encoding='utf-8') as file:
    text = file.read()
    text = text.lower()

with open('cleaned.txt', 'w', encoding='utf-8') as file:
    sentences = tokenize.sent_tokenize(text)
    pattern1 = ' +'
    pattern2 = '[,.?!-/...]'
    for s in sentences:
        s = re.sub(pattern1, ' ', s)
        s = re.sub(pattern2, '', s)
        lemmas = m.lemmatize(s)
        lemmas = ''.join(lemmas)
        file.write(lemmas)


In [4]:
#Задание 2
#Обучение модели
file = 'cleaned.txt'
data = gensim.models.word2vec.LineSentence(file)
model = gensim.models.Word2Vec(data, vector_size=300, window=5, min_count=5, epochs=50)
model.init_sims(replace=True)
model_path = "dostoevsky1"

print("Saving model...")
model.wv.save_word2vec_format(model_path, binary=True)

Saving model...


In [5]:
vocab_len = len(model.wv)
vocab = vocab_len
print('Слов в модели:', vocab_len)

Слов в модели: 3775


In [6]:
words_list = list(model.wv.key_to_index)
print(sorted([w for w in words_list]))

['1', '10', '11', '12', '13', '14', '144', '149', '15', '151', '152', '155', '16', '164', '17', '18', '1860х', '1861', '1862', '1863', '1864', '1865', '18651866', '1866', '1867', '1869', '19', '2', '20', '21', '22', '23', '24', '268', '28', '3', '30', '4', '44', '5', '6', '7', '8', '9', ':', '<…>', 'a', 'du', 'h', 'i', 'ii', 'iii', 'iv', 'la', 'm', 'nb', 'v', 'vi', 'vii', 'xix', 'xxviii', '~', '«cinq', '«а', '«ба', '«боже', '«быть', '«в', '«весь', '«вот', '«время»', '«господи', '«да', '«если', '«записка', '«и', '«идея»', '«иисус', '«как', '«на', '«не', '«нет', '«но', '«ну', '«о', '«от', '«потому', '«преступление', '«русский', '«современник»', '«среда', '«становиться', '«так', '«что', '«эпоха»', '«это', '«я', '»', 'а', 'ааа', 'август', 'авдотья', 'австрийский', 'автор', 'авторский', 'адрес', 'аккуратно', 'александр', 'александровна', 'алена', 'али', 'аль', 'амалия', 'америка', 'ан', 'анализ', 'аналогичный', 'ангел', 'английский', 'андрей', 'аппетит', 'апрель', 'арестант', 'арестовывать'

In [7]:
#Задание 3.1
#а) Выведем 5 ближайших слов для частотного союза 'но':
model.wv.most_similar("но", topn=5)

[('догадываться', 0.28748011589050293),
 ('понимать', 0.26065951585769653),
 ('скольконибудь', 0.26021572947502136),
 ('обличать', 0.2584097981452942),
 ('стоить', 0.2424881011247635)]

In [8]:
#Задание 3.1
#б) Выведем 5 ближайших слов для абстрактного понятия 'преступление':
model.wv.most_similar("преступление", topn=5)

[('преступник', 0.6506912708282471),
 ('процесс', 0.6113647222518921),
 ('анализ', 0.5571155548095703),
 ('писарев', 0.5448775291442871),
 ('нравственный', 0.5363906025886536)]

In [9]:
#Задание 3.1
#в) Выведем 5 ближайших слов для имени главного героя 'родион':
model.wv.most_similar("родион", topn=5)

[('романыч', 0.8130103349685669),
 ('романович', 0.782865047454834),
 ('батюшка', 0.5994908213615417),
 ('голубчик', 0.5949932932853699),
 ('знать:', 0.5396819114685059)]

In [10]:
#Задание 3.1
#г) Выведем 5 ближайших слов для прилагательных 'обыкновенный' и 'необыкновенный':
model.wv.most_similar(positive=['обыкновенный', 'необыкновенный'], topn=5)

[('неясный', 0.6269176602363586),
 ('неожиданный', 0.5961791276931763),
 ('впечатление', 0.5466610193252563),
 ('неприятный', 0.5460215210914612),
 ('случайный', 0.5436719059944153)]

In [17]:
#Задание 3.2
#Найдем семантическую пропорцию слов "необыкновенный" и "неожиданный"
print(model.wv.most_similar(positive=["необыкновенный", "обыкновенный"], negative=['неожиданный'])[0])

('существовать', 0.413275808095932)


In [18]:
#Задание 3.3
#Найдем лишнее в списке слово
model.wv.doesnt_match("солнце жизнь смерть".split())

'солнце'

In [35]:
#Задание 4.1
words = ["родион", "соня", "дуня", "разумихин", "свидригайлов"]
#X = model[words]
model.wv.init_sims(words)

In [38]:
pca = PCA(n_components=2)
pca.fit(model[list(model.wv)])
coords = pca.transform(model[words])

KeyError: "Key '3775' not present"

In [ ]:
plt.scatter(coords[:, 0], coords[:, 1], color='red')
plt.title('Words')

for i, word in enumerate(words):
    plt.annotate(word, xy=(coords[i, 0], coords[i, 1]))
plt.show()

In [ ]:
#Задание 4.2
things = ["топор", "диван", "обои", "подсвечник", "шляпа"]
X = model_lw[things]

In [ ]:
pca = PCA(n_components=2)
pca.fit(model[list(model.wv.vocab)])
coords = pca.transform(model[thungs])

In [ ]:
plt.scatter(coords[:, 0], coords[:, 1], color='orange')
plt.title('Things')

for i, animal in enumerate(things):
    plt.annotate(things, xy=(coords[i, 0], coords[i, 1]))
plt.show()

In [ ]:
#Задание 4.3
parts = ['о', 'об', 'но', 'то']
X = model[parts]

In [ ]:
pca = PCA(n_components=2)
pca.fit(model[list(model.wv.vocab)])
coords = pca.transform(model[parts])

In [ ]:
plt.scatter(coords[:, 0], coords[:, 1], color='green')
plt.title('Parts')

for i, part in enumerate(parts):
    plt.annotate(part, xy=(coords[i, 0], coords[i, 1]))
plt.show()

In [39]:
#Задание 5
#Возьмем любое предложение из текста и заменим каждое слово на ближайшее по вектору. 
#Для этого преобразуем предложение в нужный нам формат:
with open('dostoevsky.txt', encoding='utf-8') as file:
    text = file.read()
    sentences = tokenize.sent_tokenize(text)
    sentence = sentences[40].lower()
    s = re.sub(' +', ' ', sentence)
    s = re.sub('[,.?!-/...]', '', s)
    lemmas = m.lemmatize(s)
    sentence = ''.join(lemmas)
    print(sentence)

«я так и знать



In [40]:
#Добавим в один список ближайшие по векторам слова, если таких нет, то оставим прочерк:
new_sen = []
for word in sentence.split():
    if word in model.wv.key_to_index:
        for i in model.most_similar(positive=[word], topn=1):
            # слово + коэффициент косинусной близости
            new_word = i[0]
        new_sen.append(new_word)
    else:
        new_sen.append('-')
    
s = ' '.join(new_sen)
s

AttributeError: 'Word2Vec' object has no attribute 'most_similar'

In [41]:
#Посмотрим на изначальное предложение и конечное
print(sentence, '|\n', 'V\n', s)

«я так и знать
 |
 V
 «я так и знал
